In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D

In [2]:
# 該該函數將序列數據分割成樣本
def split_sequence(sequence, sw_width, n_features):
    '''
    這個簡單的示例，通過for循環實現有重疊截取數據，滑動步長爲1，滑動窗口寬度爲sw_width。
    以後的文章，會介紹使用yield方法來實現特定滑動步長的滑動窗口的實例。
    '''
    X, y = [], []
    
    for i in range(len(sequence)):
        # 獲取單個樣本中最後一個元素的索引，因爲python切片前閉後開，索引從0開始，所以不需要-1
        end_element_index = i + sw_width
        # 如果樣本最後一個元素的索引超過了序列索引的最大長度，說明不滿足樣本元素個數，則這個樣本丟棄
        if end_element_index > len(sequence) - 1:
            break
        # 通過切片實現步長爲1的滑動窗口截取數據組成樣本的效果
        seq_x, seq_y = sequence[i:end_element_index], sequence[end_element_index]
        
        X.append(seq_x)
        y.append(seq_y)
        
        process_X, process_y = np.array(X), np.array(y)
        process_X = process_X.reshape((process_X.shape[0], process_X.shape[1], n_features))
    
    print('split_sequence:\nX:\n{}\ny:\n{}\n'.format(np.array(X), np.array(y)))
    print('X_shape:{},y_shape:{}\n'.format(np.array(X).shape, np.array(y).shape))
    print('train_X:\n{}\ntrain_y:\n{}\n'.format(process_X, process_y))
    print('train_X.shape:{},trian_y.shape:{}\n'.format(process_X.shape, process_y.shape))
    return process_X, process_y

In [3]:
def oned_cnn_model(sw_width, n_features, X, y, test_X, epoch_num, verbose_set):
    model = Sequential()
    
    # 對於一維卷積來說，data_format='channels_last'是默認配置，該API的規則如下：
    # 輸入形狀爲：(batch, steps, channels)；輸出形狀爲：(batch, new_steps, filters)，padding和strides的變化會導致new_steps變化
    # 如果設置爲data_format = 'channels_first'，則要求輸入形狀爲： (batch, channels, steps).
    model.add(Conv1D(filters=64, kernel_size=2, activation='relu',
                     strides=1, padding='valid', data_format='channels_last',
                     input_shape=(sw_width, n_features)))
    
    # 對於一維池化層來說，data_format='channels_last'是默認配置，該API的規則如下：
    # 3D 張量的輸入形狀爲: (batch_size, steps, features)；輸出3D張量的形狀爲：(batch_size, downsampled_steps, features)
    # 如果設置爲data_format = 'channels_first'，則要求輸入形狀爲：(batch_size, features, steps)
    model.add(MaxPooling1D(pool_size=2, strides=None, padding='valid', 
                           data_format='channels_last')) 
    
    # data_format參數的作用是在將模型從一種數據格式切換到另一種數據格式時保留權重順序。默認爲channels_last。
    # 如果設置爲channels_last，那麼數據輸入形狀應爲：（batch，…，channels）；如果設置爲channels_first，那麼數據輸入形狀應該爲（batch，channels，…）
    # 輸出爲（batch, 之後參數尺寸的乘積）
    model.add(Flatten())
    
    # Dense執行以下操作：output=activation（dot（input，kernel）+bias），
    # 其中,activation是激活函數，kernel是由層創建的權重矩陣，bias是由層創建的偏移向量（僅當use_bias爲True時適用）。
    # 2D 輸入：(batch_size, input_dim)；對應 2D 輸出：(batch_size, units)
    model.add(Dense(units=50, activation='relu',
                use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros',))
    
    # 因爲要預測下一個時間步的值，因此units設置爲1
    model.add(Dense(units=1))
    
    # 配置模型
    model.compile(optimizer='adam', loss='mse',
                 metrics=['accuracy'], loss_weights=None, sample_weight_mode=None, weighted_metrics=None, target_tensors=None)
    
    print('\n',model.summary())
    # X爲輸入數據，y爲數據標籤；batch_size：每次梯度更新的樣本數，默認爲32。
    # verbose: 0,1,2. 0=訓練過程無輸出，1=顯示訓練過程進度條，2=每訓練一個epoch打印一次信息
    
    history = model.fit(X, y, batch_size=32, epochs=epoch_num, verbose=verbose_set)
    
    
    yhat = model.predict(test_X, verbose=0)
    print('\nyhat:', yhat)
    
    return model, history

In [4]:
train_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
sw_width = 3
n_features = 1
epoch_num = 1000
verbose_set = 0

train_X, train_y = split_sequence(train_seq, sw_width, n_features)

# 預測
x_input = np.array([70, 80, 90])
x_input = x_input.reshape((1, sw_width, n_features))

model, history = oned_cnn_model(sw_width, n_features, train_X, train_y, x_input, epoch_num, verbose_set)

print('\ntrain_acc:%s'%np.mean(history.history['accuracy']), '\ntrain_loss:%s'%np.mean(history.history['loss']))

split_sequence:
X:
[[10 20 30]
 [20 30 40]
 [30 40 50]
 [40 50 60]
 [50 60 70]
 [60 70 80]]
y:
[40 50 60 70 80 90]

X_shape:(6, 3),y_shape:(6,)

train_X:
[[[10]
  [20]
  [30]]

 [[20]
  [30]
  [40]]

 [[30]
  [40]
  [50]]

 [[40]
  [50]
  [60]]

 [[50]
  [60]
  [70]]

 [[60]
  [70]
  [80]]]
train_y:
[40 50 60 70 80 90]

train_X.shape:(6, 3, 1),trian_y.shape:(6,)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 2, 64)             192       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1, 64)            0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                                 
 dense (Dense)      